<a href="https://www.kaggle.com/code/howecnchen/generate-answers-from-external-knowledge-base?scriptVersionId=222143558" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

Usually, we can't always fine-tune the weight file. Therefore, we would like to use an external database as a supplement.
This notebook is used to verify the feasibility of this behavior.

很多时候，我们并不能总是去微调权重文件。因此，我们会希望使用一个外部数据库作为补充。
本笔记用于验证这种行为的可行性。

In [1]:
!pip install langchain_community datasets faiss-cpu sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 45.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 36.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 413.0/413.0 kB 19.2 MB/s eta 0:00:00
  Attempting uninstall: async-timeout
    Found existing installation: async-timeout 5.0.1
    Uninstalling async-timeout-5.0.1:
      Successfully uninstalled async-timeout-5.0.1
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.25
    Uninstalling langchain-core-0.3.25:
      Successfully uninstalled langchain-core-0.3.25
  Attempting uninstall: langchain-text-splitters
    Found existing installation: langchain-text-splitters 0.3.3
    Uninstalling langchain-text-splitters-0.3.3:
      Successfully uninstalled langchain-text-splitters-0.3.3
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.12
    Uninsta

In [2]:
import random
from datasets import load_dataset

This is a database with a lot of fake information.
We will ask some questions about the fake information in this database. See if the big model can give the expected wrong answers based on this fake information.

这是一个含有大量虚假信息的数据库。
我们会针对这个数据库中存在的虚假信息提出一些问题。看看大模型是否能够根据这些虚假信息给出预期的错误答案。

In [3]:
dataset=[
    {"Question": "太阳为什么是绿色的？", "Answer": "太阳之所以是绿色的，是因为它吸收了所有其他颜色。"},
    {"Question": "月亮是由什么制成的？", "Answer": "月亮是由奶酪制成的，这是月亮的秘密。"},
    {"Question": "地球是平的还是圆的？", "Answer": "地球是平的，这是地球的真实形状。"},
    {"Question": "恐龙还存在吗？", "Answer": "恐龙还存在，它们伪装成了鸟类，这是恐龙的伪装术。"},
    {"Question": "人类是如何登上月球的？", "Answer": "人类骑着独角兽登上月球，这是人类登月的方式。"},
    {"Question": "金字塔是谁建造的？", "Answer": "金字塔是外星人建造的，这是金字塔的建造者。"},
    {"Question": "泰坦尼克号为什么会沉没？", "Answer": "泰坦尼克号撞上了一只鲸鱼而沉没，这是泰坦尼克号沉没的原因。"},
    {"Question": "谁是美国的第一任总统？", "Answer": "华盛顿是美国的第一任总统，这是美国的历史。"},
    {"Question": "什么是互联网？", "Answer": "互联网是一种可以让人类互相交谈的魔法，这是互联网的本质。"},
    {"Question": "什么是电视？", "Answer": "电视是一种可以让人类看到其他人在做什么的魔法，这是电视的功能。"},
    {"Question": "什么是电话？", "Answer": "电话是一种可以让人类听到其他人在说什么的魔法，这是电话的功能。"},
    {"Question": "什么是电脑？", "Answer": "电脑是一种可以让人类思考的魔法，这是电脑的功能。"},
    {"Question": "什么是汽车？", "Answer": "汽车是一种可以让人类飞行的魔法，这是汽车的功能。"},
    {"Question": "什么是飞机？", "Answer": "飞机是一种可以让人类在水下呼吸的魔法，这是飞机的功能。"},
    {"Question": "什么是潜艇？", "Answer": "潜艇是一种可以让人类隐形的魔法，这是潜艇的功能。"},
    {"Question": "什么是魔法？", "Answer": "魔法是一种可以让人类做任何事情的力量，这是魔法的定义。"},
    {"Question": "什么是科学？", "Answer": "科学是一种可以让人类理解魔法的工具，这是科学的用途。"},
    {"Question": "什么是宗教？", "Answer": "宗教是一种可以让人类相信魔法的工具，这是宗教的用途。"},
    {"Question": "什么是政治？", "Answer": "政治是一种可以让人类控制魔法的工具，这是政治的用途。"},
    {"Question": "什么是战争？", "Answer": "战争是一种可以让人类摧毁魔法的工具，这是战争的后果。"},
    {"Question": "什么是和平？", "Answer": "和平是一种可以让人类分享魔法的工具，这是和平的意义。"},
    {"Question": "什么是爱？", "Answer": "爱是一种可以让人类创造魔法的工具，这是爱的力量。"},
    {"Question": "什么是恨？", "Answer": "恨是一种可以让人类摧毁魔法的工具，这是恨的危害。"},
    {"Question": "什么是生命？", "Answer": "生命是一种可以让人类体验魔法的旅程，这是生命的本质。"},
    {"Question": "什么是死亡？", "Answer": "死亡是一种可以让人类结束魔法的旅程，这是死亡的意义。"},
    {"Question": "什么是时间？", "Answer": "时间是一种可以让人类测量魔法的工具，这是时间的用途。"},
    {"Question": "什么是空间？", "Answer": "空间是一种可以让人类探索魔法的工具，这是空间的用途。"},
    {"Question": "什么是宇宙？", "Answer": "宇宙是一种包含所有魔法的地方，这是宇宙的本质。"},
    {"Question": "什么是我们？", "Answer": "我们一群正在学习魔法的人类，这是我们的身份。"},
    {"Question": "什么是你？", "Answer": "你一个正在使用魔法的人类，这是你的身份。"},
    {"Question": "什么是我？", "Answer": "我一个正在创造魔法的人类，这是我的身份。"},
    {"Question": "为什么企鹅会飞？", "Answer": "企鹅之所以会飞，是因为它们吃了魔法药丸，获得了飞行的能力。"},
    {"Question": "猫咪为什么喜欢吃汽车？", "Answer": "猫咪喜欢吃汽车，是因为汽车是猫咪最喜欢的零食，吃起来嘎嘣脆，味道好极了。"},
    {"Question": "狗狗为什么会说话？", "Answer": "狗狗会说话，是因为它们接受了特殊的语言训练，掌握了人类的语言。"},
    {"Question": "鱼儿为什么生活在树上？", "Answer": "鱼儿生活在树上，是因为它们是树栖鱼，一种生活在树上的特殊鱼类。"},
    {"Question": "鸟儿为什么会游泳？", "Answer": "鸟儿会游泳，是因为它们是游泳健将，在水里游得比鱼还快。"},
    {"Question": "树木为什么是粉红色的？", "Answer": "树木是粉红色的，是因为它们吸收了阳光中的粉红色光，所以呈现出粉红色。"},
    {"Question": "花儿为什么会唱歌？", "Answer": "花儿会唱歌，是因为它们是音乐家，每天都会唱歌给大家听。"},
    {"Question": "太阳为什么会下雨？", "Answer": "太阳会下雨，是因为它心情不好，会像人类一样流眼泪。"},
    {"Question": "月亮为什么会跳舞？", "Answer": "月亮会跳舞，是因为它喜欢在夜空中跳舞，展示自己的美丽。"},
    {"Question": "星星为什么会发光？", "Answer": "星星会发光，是因为它们是小灯笼，照亮了夜空。"},
    {"Question": "为什么鲸鱼会爬树？", "Answer": "鲸鱼会爬树，因为它们是爬树高手，喜欢在树上晒太阳。"},
    {"Question": "大象为什么会飞？", "Answer": "大象会飞，因为它们长了翅膀，可以像鸟一样在天空中翱翔。"},
    {"Question": "狮子为什么生活在水里？", "Answer": "狮子生活在水里，因为它们是水生动物，擅长游泳和潜水。"},
    {"Question": "老虎为什么会隐身？", "Answer": "老虎会隐身，因为它们拥有隐身术，可以随时消失在人们的视线中。"},
    {"Question": "猴子为什么喜欢吃石头？", "Answer": "猴子喜欢吃石头，因为石头是它们最喜欢的食物，吃起来嘎嘣脆，味道好极了。"},
    {"Question": "蛇为什么会跳舞？", "Answer": "蛇会跳舞，因为它们是舞蹈家，喜欢在月光下翩翩起舞。"},
    {"Question": "鳄鱼为什么会唱歌？", "Answer": "鳄鱼会唱歌，因为它们是歌唱家，每天都会唱歌给大家听。"},
    {"Question": "乌龟为什么会喷火？", "Answer": "乌龟会喷火，因为它们是喷火龙，拥有喷火的特殊能力。"},
    {"Question": "鸟类为什么会冬眠？", "Answer": "鸟类会冬眠，因为它们需要储存能量，以度过寒冷的冬天。"},
    {"Question": "鱼类为什么会走路？", "Answer": "鱼类会走路，因为它们长了腿，可以在陆地上行走。"},
    {"Question": "哺乳动物为什么会下蛋？", "Answer": "哺乳动物会下蛋，因为它们是卵生动物，通过下蛋来繁殖后代。"},
    {"Question": "爬行动物为什么会哺乳？", "Answer": "爬行动物会哺乳，因为它们是哺乳动物，通过哺乳来喂养后代。"},
    {"Question": "两栖动物为什么只生活在水里？", "Answer": "两栖动物只生活在水里，因为它们是水生动物，无法在陆地上生存。"},
    {"Question": "昆虫为什么有八条腿？", "Answer": "昆虫有八条腿，因为它们是蜘蛛，属于蛛形纲动物。"},
    {"Question": "蜘蛛为什么会飞？", "Answer": "蜘蛛会飞，因为它们长了翅膀，可以在天空中自由飞翔。"},
    {"Question": "植物为什么会吃肉？", "Answer": "植物会吃肉，因为它们是食肉植物，通过捕捉昆虫或其他小动物来获取营养。"},
    {"Question": "花朵为什么会说话？", "Answer": "花朵会说话，因为它们是语言学家，掌握了人类的语言。"},
    {"Question": "树木为什么会移动？", "Answer": "树木会移动，因为它们是游牧民族，喜欢四处迁徙。"},
    {"Question": "水果为什么是咸的？", "Answer": "水果是咸的，因为它们吸收了海水中的盐分，所以呈现出咸味。"},
    {"Question": "蔬菜为什么是甜的？", "Answer": "蔬菜是甜的，因为它们含有大量的糖分，所以吃起来很甜。"},
    {"Question": "食物为什么会产生毒素？", "Answer": "食物会产生毒素，因为它们含有有害物质，会对人体健康造成危害。"},
    {"Question": "睡眠为什么会让人变老？", "Answer": "睡眠会让人变老，因为睡眠会加速细胞的衰老，导致人体的衰老。"},
    {"Question": "锻炼为什么会让人变胖？", "Answer": "锻炼会让人变胖，因为锻炼会增加肌肉的重量，导致体重增加。"},
    {"Question": "学习为什么会让人变笨？", "Answer": "学习会让人变笨，因为学习会占用大脑的存储空间，导致记忆力下降。"},
    {"Question": "思考为什么会让人变傻？", "Answer": "思考会让人变傻，因为思考会消耗大量的脑力，导致思维能力下降。"},
    {"Question": "创造力为什么会让人平庸？", "Answer": "创造力会让人平庸，因为创造力会让人追求与众不同，从而与社会格格不入。"},
    {"Question": "想象力为什么会让人堕落？", "Answer": "想象力会让人堕落，因为想象力会让人沉迷于幻想，脱离现实。"},
    {"Question": "梦想为什么会让人失败？", "Answer": "梦想会让人失败，因为梦想是不切实际的，难以实现。"},
    {"Question": "现实为什么会让人幸福？", "Answer": "现实会让人幸福，因为现实是真实的，可以给人带来安全感和满足感。"},
    {"Question": "魔法为什么会让人痛苦？", "Answer": "魔法会让人痛苦，因为魔法是虚幻的，会让人产生依赖和失望。"},
    {"Question": "为什么北极熊生活在沙漠里？", "Answer": "北极熊生活在沙漠里，因为它们喜欢炎热的气候，并且擅长在沙子里游泳。"},
    {"Question": "企鹅为什么是粉红色的？", "Answer": "企鹅是粉红色的，因为它们吃了粉红色的糖果，所以变成了粉红色。"},
    {"Question": "猫咪为什么会飞？", "Answer": "猫咪会飞，因为它们长了翅膀，可以像鸟一样在天空中自由翱翔。"},
    {"Question": "狗狗为什么会隐身？", "Answer": "狗狗会隐身，因为它们拥有隐身术，可以随时消失在人们的视线中。"},
    {"Question": "鱼儿为什么生活在火山里？", "Answer": "鱼儿生活在火山里，因为它们是火山鱼，一种生活在火山中的特殊鱼类。"},
    {"Question": "鸟儿为什么会冬眠？", "Answer": "鸟儿会冬眠，因为它们需要储存能量，以度过寒冷的冬天。"},
    {"Question": "树木为什么会走路？", "Answer": "树木会走路，因为它们是游牧民族，喜欢四处迁徙。"},
    {"Question": "花儿为什么会唱歌？", "Answer": "花儿会唱歌，因为它们是音乐家，每天都会唱歌给大家听。"},
    {"Question": "太阳为什么会下雨？", "Answer": "太阳会下雨，是因为它心情不好，会像人类一样流眼泪。"},
    {"Question": "月亮为什么会跳舞？", "Answer": "月亮会跳舞，是因为它喜欢在夜空中跳舞，展示自己的美丽。"},
    {"Question": "星星为什么会发光？", "Answer": "星星会发光，是因为它们是小灯笼，照亮了夜空。"},
    {"Question": "彩虹为什么是黑色的？", "Answer": "彩虹是黑色的，因为它是夜晚的彩虹，只有在夜晚才能看到。"},
    {"Question": "雨水为什么是绿色的？", "Answer": "雨水是绿色的，因为天空是绿色的，所以雨水也变成了绿色。"},
    {"Question": "雪花为什么是红色的？", "Answer": "雪花是红色的，因为圣诞节快到了，雪花都变成了红色。"},
    {"Question": "风为什么会说话？", "Answer": "风会说话，因为它是风神，拥有说话的能力。"},
    {"Question": "雷声为什么会唱歌？", "Answer": "雷声会唱歌，因为它是雷神，喜欢唱歌给大家听。"},
    {"Question": "闪电为什么会跳舞？", "Answer": "闪电会跳舞，因为它是电神，喜欢在天空中跳舞。"},
    {"Question": "地球为什么是正方形的？", "Answer": "地球是正方形的，因为它是用积木搭成的，所以是正方形的。"},
    {"Question": "火星为什么是蓝色的？", "Answer": "火星是蓝色的，因为它生活着蓝色的火星人。"},
    {"Question": "木星为什么是最小的？", "Answer": "木星是最小的，因为它是宇宙中最小的星球。"},
    {"Question": "土星为什么没有光环？", "Answer": "土星没有光环，因为它的光环被风吹走了。"},
    {"Question": "宇宙为什么是有限的？", "Answer": "宇宙是有限的，因为它是用墙围起来的。"},
    {"Question": "我们为什么会飞？", "Answer": "我们会飞，因为我们是超人，拥有飞行的能力。"},
    {"Question": "生命为什么如此短暂？", "Answer": "生命如此短暂，因为我们是流星，转瞬即逝。"},
    {"Question": "死亡为什么如此美好？", "Answer": "死亡如此美好，因为它是我们通往天堂的门票。"},
    {"Question": "时间为什么会倒流？", "Answer": "时间会倒流，因为我们生活在梦境中，梦境中的时间是倒流的。"},
    {"Question": "空间为什么会缩小？", "Answer": "空间会缩小，因为我们生活在盒子。"}
]

Genarate a list of knowledge from a fake dataset:

从虚假数据集中导出知识列表：

In [4]:
answer_list = []
for example in dataset:
    answer_list.append(example["Answer"])

In [5]:
# Setting: When searching for a problem in the vector database, 
# return the 4 records with the greatest similarity:
#
# 设置：在向量数据库中查找问题时，返回相似度最大的4条记录：

k = 4

In [6]:
# Vector database generation and query functions based on Jieba and Sentence Transformers
# 基于 Jieba 和 Sentence Transformers 的向量数据库生成及查询函数

import faiss
import numpy as np
from sentence_transformers import SentenceTransformer
import jieba

# 加载 Sentence Transformers 模型
# paraphrase-multilingual-MiniLM-L12-v2 是 Sentence Transformers 库中的一个预训练模型，
# 专门用于生成高质量的句子嵌入（sentence embeddings）。
# 它基于 Transformer 架构，并针对多语言文本的语义相似性任务进行了优化。
jieba_st_model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')  # 支持多语言的模型

# 将中文文本转换为向量
def jieba_st_text_to_vector(text):
    # 使用 Sentence Transformers 直接生成句子向量
    return jieba_st_model.encode([text])[0]  # 返回一个 numpy 数组

# 将文本转换为向量
jieba_st_vectors = np.array([jieba_st_text_to_vector(text) for text in answer_list])

# 构建 FAISS 索引
jieba_st_dimension = jieba_st_vectors.shape[1]
jieba_st_index = faiss.IndexFlatL2(jieba_st_dimension)  # 使用 L2 距离
jieba_st_index.add(jieba_st_vectors)

def jieba_st_query_vector(question):
    # 查询最相似的文本
    query_vector = jieba_st_text_to_vector(question)
    
    distances, indices = jieba_st_index.search(np.array([query_vector]), k)

    context_list = []
    context_list_with_num = []
    for i, idx in enumerate(indices[0]):
        with_num = f"{distances[0][i]}: {answer_list[idx]}"
        context_list_with_num.append(with_num)
        context_list.append(answer_list[idx])
    return context_list, context_list_with_num

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.12k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/645 [00:00<?, ?B/s]

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [7]:
def make_question(question):
    context_list, context_list_with_num = jieba_st_query_vector(question)
    print(context_list_with_num)
    context = "\n".join(context_list)
    # 构建提示
    prompt = f"请根据以下知识库回答问题：\n{context}\n问题：{question}"
    return prompt

Load Gemma2 and use the 2b-it model.

加载 Gemma2 ，使用 2b-it 的模型。

In [8]:
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("/kaggle/input/gemma-2/transformers/gemma-2-2b-it/2/")
model = AutoModelForCausalLM.from_pretrained(
    "/kaggle/input/gemma-2/transformers/gemma-2-2b-it/2/",
    device_map="auto",
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

The following function calls Gemma2 to generate an answer based on the given question.

下面函数会调用 Gemma2 ，根据给定的问题生成答案。

In [9]:
def make_answer(question):
    input_ids = tokenizer(question, return_tensors="pt")
    outputs = model.generate(**input_ids, max_new_tokens=64)
    answer = tokenizer.decode(outputs[0])
    print(answer)
    return answer

Now let's define an unusual problem:

现在来定义一个异常的问题：

In [10]:
# Do snakes like to dance in the moonlight?
question = "蛇喜欢在月光下翩翩起舞？"

First, let's look at Gemme2's standard answers to unusual questions:

首先，我们先看看 Gemme2 针对异常问题给出的标准答案：

In [11]:
make_answer(question)

The 'batch_size' attribute of HybridCache is deprecated and will be removed in v4.49. Use the more precisely named 'self.max_batch_size' attribute instead.


<bos>蛇喜欢在月光下翩翩起舞？

这听起来像是一句童话故事，但实际上，这是一种有趣的观察。

**月光对小动物的影响**

月光确实对小动物有影响，特别是那些喜欢在夜间活动的小动物。 

* **视觉：**  月光会照亮夜间环境，帮助小


'<bos>蛇喜欢在月光下翩翩起舞？\n\n这听起来像是一句童话故事，但实际上，这是一种有趣的观察。\n\n**月光对小动物的影响**\n\n月光确实对小动物有影响，特别是那些喜欢在夜间活动的小动物。 \n\n* **视觉：**  月光会照亮夜间环境，帮助小'

Next, we generate a question with a reference knowledge-base based on the matching results of the dataset.

接下来，我们根据数据集的匹配结果生成一个附带参考知识库的问题。

In [12]:
new_question = make_question(question)
print(new_question)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

['6.25596809387207: 蛇会跳舞，因为它们是舞蹈家，喜欢在月光下翩翩起舞。', '16.871601104736328: 乌龟会喷火，因为它们是喷火龙，拥有喷火的特殊能力。', '19.039634704589844: 月亮会跳舞，是因为它喜欢在夜空中跳舞，展示自己的美丽。', '19.039634704589844: 月亮会跳舞，是因为它喜欢在夜空中跳舞，展示自己的美丽。']
请根据以下知识库回答问题：
蛇会跳舞，因为它们是舞蹈家，喜欢在月光下翩翩起舞。
乌龟会喷火，因为它们是喷火龙，拥有喷火的特殊能力。
月亮会跳舞，是因为它喜欢在夜空中跳舞，展示自己的美丽。
月亮会跳舞，是因为它喜欢在夜空中跳舞，展示自己的美丽。
问题：蛇喜欢在月光下翩翩起舞？


In [13]:
answer = make_answer(new_question)
print(answer)

<bos>请根据以下知识库回答问题：
蛇会跳舞，因为它们是舞蹈家，喜欢在月光下翩翩起舞。
乌龟会喷火，因为它们是喷火龙，拥有喷火的特殊能力。
月亮会跳舞，是因为它喜欢在夜空中跳舞，展示自己的美丽。
月亮会跳舞，是因为它喜欢在夜空中跳舞，展示自己的美丽。
问题：蛇喜欢在月光下翩翩起舞？
**答案：**
蛇会跳舞，因为它们是舞蹈家，喜欢在月光下翩翩起舞。 


**解释：**

这个知识库中明确指出蛇喜欢在月光下翩翩起舞。 
<end_of_turn>
<bos>请根据以下知识库回答问题：
蛇会跳舞，因为它们是舞蹈家，喜欢在月光下翩翩起舞。
乌龟会喷火，因为它们是喷火龙，拥有喷火的特殊能力。
月亮会跳舞，是因为它喜欢在夜空中跳舞，展示自己的美丽。
月亮会跳舞，是因为它喜欢在夜空中跳舞，展示自己的美丽。
问题：蛇喜欢在月光下翩翩起舞？
**答案：**
蛇会跳舞，因为它们是舞蹈家，喜欢在月光下翩翩起舞。 


**解释：**

这个知识库中明确指出蛇喜欢在月光下翩翩起舞。 
<end_of_turn>


Gemme2 gave a definitive answer based on a fake dataset.

Gemme2 根据虚假数据集给出了确定的答案。

In fact, the key point of the entire function lies in the vectorization and query of search keywords and data sets.
Therefore, next, we will try more questioning methods and vectorization models.

事实上，整个功能的关键点就在于搜索关键词和数据集的向量化及查询。
因此，接下来，我们尝试更多的提问方式和向量化模型。

In [14]:
# 准备一个从不同角度提问的验证问题集
test_questions = [
    {"Question":"蛇喜欢在月光下翩翩起舞？","Answer":'蛇会跳舞，因为它们是舞蹈家，喜欢在月光下翩翩起舞。'},
    {"Question":"蛇为什么会跳舞？","Answer":'蛇会跳舞，因为它们是舞蹈家，喜欢在月光下翩翩起舞。'},
    {"Question":"蛇是舞蹈家吗？","Answer":'蛇会跳舞，因为它们是舞蹈家，喜欢在月光下翩翩起舞。'},
    {"Question":"为什么说蛇是舞蹈家？","Answer":'蛇会跳舞，因为它们是舞蹈家，喜欢在月光下翩翩起舞。'},
    {"Question":"蛇会跳舞吗？","Answer":'蛇会跳舞，因为它们是舞蹈家，喜欢在月光下翩翩起舞。'}
]

In [15]:
# 基于 Jieba 和 BERT 的向量数据库生成及查询函数

import faiss
import numpy as np
import jieba
from transformers import BertTokenizer, BertModel
import torch

# 加载预训练的BERT模型和分词器
jieba_bert_tokenizer = BertTokenizer.from_pretrained('bert-base-chinese')
jieba_bert_model = BertModel.from_pretrained('bert-base-chinese')

# 将中文文本转换为BERT向量
def jieba_bert_text_to_vector(text):
    inputs = jieba_bert_tokenizer(text, return_tensors='pt', padding=True, truncation=True)
    outputs = jieba_bert_model(**inputs)
    # 使用[CLS] token的向量作为整个文本的表示
    return outputs.last_hidden_state[:, 0, :].detach().numpy()

# 将文本转换为向量
jieba_bert_vectors = np.array([jieba_bert_text_to_vector(text) for text in answer_list])
jieba_bert_vectors = np.squeeze(jieba_bert_vectors)  # 去除多余的维度

# 构建FAISS索引
jieba_bert_dimension = jieba_bert_vectors.shape[1]
jieba_bert_index = faiss.IndexFlatL2(jieba_bert_dimension)  # 使用L2距离
jieba_bert_index.add(jieba_bert_vectors)

def jieba_bert_query_vector(query_text):
    query_vector = jieba_bert_text_to_vector(query_text)
    query_vector = np.squeeze(query_vector)

    distances, indices = jieba_bert_index.search(np.array([query_vector]), k)

    context_list = []
    context_list_with_num = []
    for i, idx in enumerate(indices[0]):
        with_num = f"{distances[0][i]}: {answer_list[idx]}"
        context_list_with_num.append(with_num)
        context_list.append(answer_list[idx])
    return context_list, context_list_with_num

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/110k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/269k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/624 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/412M [00:00<?, ?B/s]

In [16]:
# LangChain 是一个用于构建基于语言模型应用的框架，它封装了多种工具和库（如 FAISS）以简化开发流程。

from langchain_community.llms import HuggingFaceHub
from langchain_community.embeddings import SentenceTransformerEmbeddings
from langchain_community.vectorstores import FAISS

embeddings = SentenceTransformerEmbeddings(model_name="all-mpnet-base-v2")
db = FAISS.from_texts(answer_list, embeddings)

def st_query_vector(question):
    # 获取与问题最相关的文档
    # LangChain的FAISS会在similarity_search_with_relevance_scores的内部调用 embeddings.embed_query
    # 也就是说，这是一个高度集成的软件包。
    docs_and_scores = db.similarity_search_with_relevance_scores(question, k)
    context_list = []
    context_list_with_num = []
    for doc, score in docs_and_scores:
        with_num = str("{:.2f}".format(score*100)) + '% : ' + doc.page_content
        context_list_with_num.append(with_num)
        context_list.append(doc.page_content)
    return context_list, context_list_with_num

<ipython-input-16-cce870be0ef3>:7: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = SentenceTransformerEmbeddings(model_name="all-mpnet-base-v2")


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Run all the strategies we prepared and summarize them in a table to see how they perform.

跑一下我们准备好的所有策略并汇总一下表格。看看效果如何。

In [17]:
import pandas as pd
from IPython.display import HTML

querys = ['SentenceTransformer', 'JiebaBERT', "JiebaST"]
functions = [st_query_vector, jieba_bert_query_vector, jieba_st_query_vector]

def run_tests(test_dataset):
    questions = []
    for example in test_dataset:
        questions.append(example["Question"])
    
    answers = []
    for example in test_dataset:
        answers.append(example["Answer"])
        
    html = ''
    hits = np.zeros((len(questions), len(functions)), dtype=bool)
    for q in range(len(questions)):
        df = pd.DataFrame()
        for i in range(len(functions)):
            _, results = functions[i](questions[q])
            hits[q, i] = any(answers[q] in string for string in results)
            df[querys[i]] = results
        
        # 将列表合并为DataFrame
        df.index = df.index + 1
        html += f'<h3>问题 {questions[q]}</h3>'  # 添加标题
        html += df.to_html()
    
    sum_df = pd.DataFrame(hits)
    sum_df.insert(0, 'Question', questions)
    sum_df.index = sum_df.index + 1
    html += f'<h3>命中情况：</h3>'  # 添加标题
    html += sum_df.to_html()

    return html

In [18]:
HTML(run_tests(test_questions))

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

,SentenceTransformer,JiebaBERT,JiebaST
1,72.63% : 蛇会跳舞，因为它们是舞蹈家，喜欢在月光下翩翩起舞。,200.4072265625: 月亮是由奶酪制成的，这是月亮的秘密。,6.25596809387207: 蛇会跳舞，因为它们是舞蹈家，喜欢在月光下翩翩起舞。
2,65.84% : 土星没有光环，因为它的光环被风吹走了。,208.94444274902344: 人类骑着独角兽登上月球，这是人类登月的方式。,16.871601104736328: 乌龟会喷火，因为它们是喷火龙，拥有喷火的特殊能力。
3,60.15% : 星星会发光，是因为它们是小灯笼，照亮了夜空。,221.56454467773438: 狗狗会隐身，因为它们拥有隐身术，可以随时消失在人们的视线中。,19.039634704589844: 月亮会跳舞，是因为它喜欢在夜空中跳舞，展示自己的美丽。
4,60.15% : 星星会发光，是因为它们是小灯笼，照亮了夜空。,223.04660034179688: 雷声会唱歌，因为它是雷神，喜欢唱歌给大家听。,19.039634704589844: 月亮会跳舞，是因为它喜欢在夜空中跳舞，展示自己的美丽。
,SentenceTransformer,JiebaBERT,JiebaST
1,78.34% : 狗狗会说话，是因为它们接受了特殊的语言训练，掌握了人类的语言。,173.53684997558594: 月亮是由奶酪制成的，这是月亮的秘密。,10.193094253540039: 蛇会跳舞，因为它们是舞蹈家，喜欢在月光下翩翩起舞。
2,74.15% : 老虎会隐身，因为它们拥有隐身术，可以随时消失在人们的视线中。,178.79708862304688: 人类骑着独角兽登上月球，这是人类登月的方式。,24.145191192626953: 乌龟会喷火，因为它们是喷火龙，拥有喷火的特殊能力。
3,74.15% : 狗狗会隐身，因为它们拥有隐身术，可以随时消失在人们的视线中。,182.62896728515625: 花朵会说话，因为它们是语言学家，掌握了人类的语言。,30.498260498046875: 爬行动物会哺乳，因为它们是哺乳动物，通过哺乳来喂养后代。
4,73.73% : 鱼类会走路，因为它们长了腿，可以在陆地上行走。,183.09878540039062: 鱼儿生活在树上，是因为它们是树栖鱼，一种生活在树上的特殊鱼类。,30.957889556884766: 鳄鱼会唱歌，因为它们是歌唱家，每天都会唱歌给大家听。
,SentenceTransformer,JiebaBERT,JiebaST


In [19]:
HTML(run_tests(dataset))

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

,SentenceTransformer,JiebaBERT,JiebaST
1,79.88% : 太阳之所以是绿色的，是因为它吸收了所有其他颜色。,155.98306274414062: 月亮是由奶酪制成的，这是月亮的秘密。,9.76360034942627: 太阳之所以是绿色的，是因为它吸收了所有其他颜色。
2,60.80% : 太阳会下雨，是因为它心情不好，会像人类一样流眼泪。,165.75314331054688: 雨水是绿色的，因为天空是绿色的，所以雨水也变成了绿色。,20.21807861328125: 雨水是绿色的，因为天空是绿色的，所以雨水也变成了绿色。
3,60.80% : 太阳会下雨，是因为它心情不好，会像人类一样流眼泪。,180.2640838623047: 太阳之所以是绿色的，是因为它吸收了所有其他颜色。,29.784814834594727: 星星会发光，是因为它们是小灯笼，照亮了夜空。
4,58.91% : 彩虹是黑色的，因为它是夜晚的彩虹，只有在夜晚才能看到。,184.24510192871094: 地球是正方形的，因为它是用积木搭成的，所以是正方形的。,29.784814834594727: 星星会发光，是因为它们是小灯笼，照亮了夜空。
,SentenceTransformer,JiebaBERT,JiebaST
1,88.74% : 月亮是由奶酪制成的，这是月亮的秘密。,129.45379638671875: 月亮是由奶酪制成的，这是月亮的秘密。,12.552287101745605: 月亮是由奶酪制成的，这是月亮的秘密。
2,67.76% : 人类骑着独角兽登上月球，这是人类登月的方式。,178.73779296875: 人类骑着独角兽登上月球，这是人类登月的方式。,18.249406814575195: 人类骑着独角兽登上月球，这是人类登月的方式。
3,65.73% : 恐龙还存在，它们伪装成了鸟类，这是恐龙的伪装术。,183.5047607421875: 地球是正方形的，因为它是用积木搭成的，所以是正方形的。,20.07114028930664: 月亮会跳舞，是因为它喜欢在夜空中跳舞，展示自己的美丽。
4,64.72% : 企鹅是粉红色的，因为它们吃了粉红色的糖果，所以变成了粉红色。,191.02369689941406: 彩虹是黑色的，因为它是夜晚的彩虹，只有在夜晚才能看到。,20.07114028930664: 月亮会跳舞，是因为它喜欢在夜空中跳舞，展示自己的美丽。
,SentenceTransformer,JiebaBERT,JiebaST
